# Exercise 3

* To complete this exercise you just need to add *two* short lines of code 
  inside the training loop, where it says "YOUR CODE HERE".


In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

RANDOM_SEED = 1
torch.manual_seed(RANDOM_SEED)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
use_cpu = False

if use_cpu and torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print("Using cuda.")
else:
    DEVICE = torch.device('cpu')
    print("Using cpu.")

In [ ]:
df = pd.read_csv('./data/char74k-digits-and-uppercase.csv')
df.sample(3)

In [ ]:
LABELS = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
sample = df.sample()
print(LABELS[sample.label.iloc[0]])

image = sample.drop('label', axis=1).values.reshape((30,40,))

plt.imshow(image, cmap='gray');

In [ ]:
y_labels = df.label.values
X_images = df.drop('label', axis=1).values


In [ ]:
from sklearn.model_selection import train_test_split

# 55 images per class and 36 classes
# take 10 images from each class, so test_size = 360.

X_train, X_validation, y_train, y_validation = train_test_split(X_images,
                                                                y_labels,
                                                                test_size=360,
                                                                stratify=y_labels,
                                                                shuffle=True,
                                                                random_state=RANDOM_SEED
                                                               )

In [ ]:
X_train = torch.from_numpy(X_train).to(DEVICE, torch.float32)
y_train = torch.from_numpy(y_train).to(DEVICE, torch.long)

X_validation = torch.from_numpy(X_validation).to(DEVICE, torch.float32)
y_validation = torch.from_numpy(y_validation).to(DEVICE, torch.long)

In [ ]:
NUM_FEATURES = 1200
NUM_CLASSES = 36

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.fc1 = nn.Linear(NUM_FEATURES, 600) 
        self.fc2 = nn.Linear(600, 200)
        self.fc3 = nn.Linear(200, NUM_CLASSES)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)
        return x
    
    
classifier = Classifier()
classifier = classifier.to(DEVICE)

print(classifier)

In [ ]:
LEARNING_RATE = 0.1

minimizer = torch.optim.SGD(classifier.parameters(), lr=LEARNING_RATE)

In [ ]:
NUM_EPOCHS = 300

losses = []
accuracies = []

from sklearn.metrics import accuracy_score, confusion_matrix

for epoch in range(NUM_EPOCHS):
    minimizer.zero_grad()
    
    y_scores = classifier(X_train)

    y_log_probs = F.log_softmax(y_scores, dim=1)
    
    loss = F.nll_loss(y_log_probs, y_train)
    losses.append(loss)
        
    # YOUR CODE HERE 
    # You need to add two lines here.  
    # 
    # 1. the first line will trigger PyTorch to calculate the gradients.
    # 2. The second line will update the weights in the model.
    # 

    if not epoch % 20:
        print(f"epoch {epoch} finished, loss: {loss.item()}")


In [ ]:
plt.plot(losses);

In [ ]:
y_scores = classifier(X_validation)

_, y_predictions = torch.max(y_scores.data, 1)

accuracy = accuracy_score(y_validation, y_predictions)
print(f"Final accuracy score: {100*accuracy:.1f}%")

cm = confusion_matrix(y_validation, y_predictions)

In [ ]:
import seaborn as sns

plt.figure(figsize=(12,12))

heatmap = sns.heatmap(cm, cmap='viridis');
heatmap.set(xticklabels=LABELS);
heatmap.set(yticklabels=LABELS);